In [ ]:
from tqdm import tqdm
from PIL import Image
import pandas as pd
import os
import numpy as np


def load_data(folder):
    images = []
    for file in os.listdir(folder):
        file_id = file.replace('.png', '')
        image = Image.open(
            os.path.join(folder, file)
        ).convert('LA').resize((256, 256))
        arr = np.array(image)
        images.append(
            (int(file_id), arr)
        )
    images.sort(key=lambda i: i[0])
    return np.array([v for _id, v in images])



x_train = load_data('train')
y_train = pd.read_csv('y_train.csv')['infection']

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, Input

def build():
    img_in = Input(shape=(256, 256, 2))
    flattened = Flatten()(img_in)
    fc1 = Dense(64)(flattened)
    #fc1 = Dropout(0.3)(fc1)
    fc2 = Dense(32)(fc1)
    #fc2 = Dropout(0.3)(fc2)
    output = Dense(1, activation = 'sigmoid')(fc2)
    model = tf.keras.Model(inputs=img_in, outputs=output)
    return model


model = build()
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['BinaryAccuracy', 'AUC']
        )

model.summary()



In [ ]:
epochs = 30
batch_size = 64

history = model.fit(x = x_train,
                    y = y_train,
                    batch_size = batch_size,
                    validation_split=0.3,
                    epochs=epochs
                    )

In [ ]:
x_test = load_data('test')

y_test = model.predict(x_test)

y_test_df = pd.DataFrame()
y_test_df['id'] = np.arange(len(y_test))
y_test_df['infection'] = y_test.astype(float)
y_test_df.to_csv('submission.csv', index=False)

